In [264]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from lazypredict.Supervised import LazyRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import PoissonRegressor, LinearRegression, LassoLarsIC, BayesianRidge, Ridge, RidgeCV, LassoLars, Lasso, OrthogonalMatchingPursuitCV, HuberRegressor, LarsCV, LassoLarsCV, LassoCV, PassiveAggressiveRegressor, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import LinearSVR, NuSVR, SVR
from sklearn.dummy import DummyRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Lars
from sklearn.linear_model import SGDRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

In [250]:
diamonds = pd.read_csv('./data/diamondsql.csv')
diamonds['volume'] = (4/3) * np.pi * diamonds['x'] * diamonds['y'] * diamonds['z']
diamonds['length_width_ratio'] = diamonds['x'] / diamonds['y']
diamonds['width_depth_ratio'] = diamonds['y'] / diamonds['z']

In [251]:
diamonds.columns

Index(['index_id', 'price', 'city_id', 'carat', 'city_id.1', 'city',
       'index_id.1', 'cut_id', 'color_id', 'clarity_id', 'cut_id.1', 'cut',
       'clarity_id.1', 'clarity', 'color_id.1', 'color', 'index_id.2', 'depth',
       'table', 'x', 'y', 'z', 'volume', 'length_width_ratio',
       'width_depth_ratio'],
      dtype='object')

In [252]:
diamonds = diamonds.drop(['index_id', 'city_id', 'city_id.1','index_id.1','cut_id', 'color_id', 'clarity_id', 'cut_id.1', 'clarity_id.1', 'color_id.1', 'index_id.2'], axis = 1)

In [253]:
diamonds.head()

,price,carat,city,cut,clarity,color,depth,table,x,y,z,volume,length_width_ratio,width_depth_ratio
0,4268,1.21,Dubai,Premium,VS2,J,62.40,58.00,6.83,6.79,4.25,825.60,1.01,1.60
1,505,0.32,Kimberly,Very Good,VS2,H,63.00,57.00,4.35,4.38,2.75,219.47,0.99,1.59
2,2686,0.71,Las Vegas,Fair,VS1,G,65.50,55.00,5.62,5.53,3.65,475.16,1.02,1.52
3,738,0.41,Kimberly,Good,SI1,D,63.80,56.00,4.68,4.72,3.00,277.59,0.99,1.57
4,4882,1.02,Dubai,Ideal,SI1,G,60.50,59.00,6.55,6.51,3.95,705.52,1.01,1.65


In [254]:
diamonds.describe()

,price,carat,depth,table,x,y,z,volume,length_width_ratio,width_depth_ratio
count,40455.00,40455.00,40455.00,40455.00,40455.00,40455.00,40455.00,40455.00,40451.00,40451.00
mean,3928.44,0.80,61.75,57.45,5.73,5.73,3.54,543.71,1.00,inf
std,3992.42,0.48,1.43,2.23,1.12,1.15,0.70,330.51,0.01,NaN
min,326.00,0.20,43.00,43.00,0.00,0.00,0.00,0.00,0.00,1.00
25%,945.00,0.40,61.00,56.00,4.71,4.72,2.91,271.81,0.99,1.60
50%,2397.00,0.70,61.80,57.00,5.69,5.71,3.52,480.41,1.00,1.62
75%,5331.00,1.04,62.50,59.00,6.54,6.54,4.04,715.63,1.01,1.64
max,18823.00,4.50,79.00,95.00,10.23,58.90,8.06,16087.46,1.62,inf


In [255]:
# Reemplazar los infinitos con NaN
diamonds.replace([np.inf, -np.inf], np.nan, inplace=True)

# Rellenar NaN con interpolación lineal
diamonds.interpolate(method='linear', inplace=True)

In [256]:
diamonds_encoded = pd.get_dummies(diamonds, columns=['city','cut','clarity','color'])


In [257]:
diamonds_encoded

,price,carat,depth,table,x,y,z,volume,length_width_ratio,width_depth_ratio,...,clarity_VS2,clarity_VVS1,clarity_VVS2,color_D,color_E,color_F,color_G,color_H,color_I,color_J
0,4268,1.21,62.40,58.00,6.83,6.79,4.25,825.60,1.01,1.60,...,True,False,False,False,False,False,False,False,False,True
1,505,0.32,63.00,57.00,4.35,4.38,2.75,219.47,0.99,1.59,...,True,False,False,False,False,False,False,True,False,False
2,2686,0.71,65.50,55.00,5.62,5.53,3.65,475.16,1.02,1.52,...,False,False,False,False,False,False,True,False,False,False
3,738,0.41,63.80,56.00,4.68,4.72,3.00,277.59,0.99,1.57,...,False,False,False,True,False,False,False,False,False,False
4,4882,1.02,60.50,59.00,6.55,6.51,3.95,705.52,1.01,1.65,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,62.70,57.00,7.10,7.04,4.43,927.52,1.01,1.59,...,False,False,False,False,False,False,True,False,False,False
40451,12615,2.02,57.10,60.00,8.31,8.25,4.73,1358.33,1.01,1.74,...,False,False,False,False,False,True,False,False,False,False
40452,5457,1.01,62.70,56.00,6.37,6.42,4.01,686.92,0.99,1.60,...,False,False,False,False,False,False,False,True,False,False
40453,456,0.33,61.90,54.30,4.45,4.47,2.76,229.97,1.00,1.62,...,False,False,False,False,False,False,False,False,False,True


In [258]:
X = diamonds_encoded.drop('price', axis=1)
y = diamonds_encoded['price']

In [259]:
X

,carat,depth,table,x,y,z,volume,length_width_ratio,width_depth_ratio,city_Amsterdam,...,clarity_VS2,clarity_VVS1,clarity_VVS2,color_D,color_E,color_F,color_G,color_H,color_I,color_J
0,1.21,62.40,58.00,6.83,6.79,4.25,825.60,1.01,1.60,False,...,True,False,False,False,False,False,False,False,False,True
1,0.32,63.00,57.00,4.35,4.38,2.75,219.47,0.99,1.59,False,...,True,False,False,False,False,False,False,True,False,False
2,0.71,65.50,55.00,5.62,5.53,3.65,475.16,1.02,1.52,False,...,False,False,False,False,False,False,True,False,False,False
3,0.41,63.80,56.00,4.68,4.72,3.00,277.59,0.99,1.57,False,...,False,False,False,True,False,False,False,False,False,False
4,1.02,60.50,59.00,6.55,6.51,3.95,705.52,1.01,1.65,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,1.34,62.70,57.00,7.10,7.04,4.43,927.52,1.01,1.59,False,...,False,False,False,False,False,False,True,False,False,False
40451,2.02,57.10,60.00,8.31,8.25,4.73,1358.33,1.01,1.74,False,...,False,False,False,False,False,True,False,False,False,False
40452,1.01,62.70,56.00,6.37,6.42,4.01,686.92,0.99,1.60,False,...,False,False,False,False,False,False,False,True,False,False
40453,0.33,61.90,54.30,4.45,4.47,2.76,229.97,1.00,1.62,False,...,False,False,False,False,False,False,False,False,False,True


In [260]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")


X_train: (32364, 42), X_test: (8091, 42), y_train: (32364,), y_test: (8091,)


In [79]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

 76%|██████████████████████████████████████████████████████████████▍                   | 32/42 [26:44<01:46, 10.63s/it]

QuantileRegressor model failed to execute
Solver interior-point is not anymore available in SciPy >= 1.11.0.


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [29:09<00:00, 41.65s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000635 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2009
[LightGBM] [Info] Number of data points in the train set: 32364, number of used features: 9
[LightGBM] [Info] Start training from score 3927.895532


In [80]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
GradientBoostingRegressor,0.89,0.89,1346.31,8.72
HistGradientBoostingRegressor,0.89,0.89,1350.66,0.52
LGBMRegressor,0.89,0.89,1354.61,0.18
RandomForestRegressor,0.88,0.88,1377.04,56.60
XGBRegressor,0.88,0.88,1392.50,0.38
ExtraTreesRegressor,0.88,0.88,1396.80,21.09
PoissonRegressor,0.88,0.88,1405.78,0.10
MLPRegressor,0.88,0.88,1406.03,23.48
BaggingRegressor,0.87,0.88,1423.77,5.60


In [271]:
%%time
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

CPU times: total: 15.6 ms
Wall time: 63.6 ms


In [272]:
mean_squared_error(y_test, y_pred)


1243782.891791083

In [273]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

1115.2501476310517

In [ ]:
diamonds_test = pd.read_csv('./data/diamonds_test.csv')
diamonds_test['volume'] = (4/3) * np.pi * diamonds_test['x'] * diamonds_test['y'] * diamonds_test['z']
diamonds_test['length_width_ratio'] = diamonds_test['x'] / diamonds_test['y']
diamonds_test['width_depth_ratio'] = diamonds_test['y'] / diamonds_test['z']
diamonds_test

In [ ]:
#diamonds_test=diamonds_test.drop('city',axis=1)

In [ ]:
diamonds_encoded_test = pd.get_dummies(diamonds_test, columns=['city','cut','clarity','color'])
diamonds_encoded_test

In [ ]:
X_prueba = diamonds_encoded_test.drop('id', axis=1)


In [ ]:
y_prueba = model_xgb.predict(X_prueba)

In [ ]:
y_prueba


In [ ]:
solution = []
for i, value in enumerate(y_prueba):
    solution.append((i,value))
    

In [ ]:
solution_df = pd.DataFrame(solution)


In [ ]:
solution_df.columns = ['id', 'price']

In [ ]:
solution_df['price'].to_csv('solution.csv')